# Ejercicio 6
El porcentaje de contenidos que están publicados cuya última edición no tiene
comentario para los usuarios que realizaron 1, >10 y >100 de las últimas ediciones

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('./WikipediaDataset/contents.csv')

In [2]:
df.head()

,title,id,namespace,revision_id,parent_revision,revision_timestamp,revisor_username,revisor_id,revisor_ip,revisor_comment
0,Wikipedia:Artículos solicitados,5,4,132533307,132516631.0,2021-01-19T18:40:34Z,Metrónomo,1708233.0,NaN,Así esta mejor
1,Andorra,7,0,138035057,138027475.0,2021-08-31T18:27:01Z,NaN,NaN,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,Añadir contenido
2,Argentina,10,0,137966826,137825165.0,2021-08-28T17:17:54Z,Traitrich,4642283.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...
3,Geografía de Andorra,15,0,137986788,133666383.0,2021-08-29T17:16:00Z,Lojwe,5338508.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...
4,Demografía de Andorra,17,0,130869554,129413188.0,2020-11-12T19:56:15Z,Jialxv,3615366.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4132164 entries, 0 to 4132163
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   title               object 
 1   id                  int64  
 2   namespace           int64  
 3   revision_id         int64  
 4   parent_revision     float64
 5   revision_timestamp  object 
 6   revisor_username    object 
 7   revisor_id          float64
 8   revisor_ip          object 
 9   revisor_comment     object 
dtypes: float64(2), int64(3), object(5)
memory usage: 315.3+ MB


In [4]:
df.count()

title                 4132160
id                    4132164
namespace             4132164
revision_id           4132164
parent_revision       2648946
revision_timestamp    4132164
revisor_username      3938877
revisor_id            3938877
revisor_ip             193286
revisor_comment       3662940
dtype: int64

## Limpiando el dataset
Para la consigna no se necesitan todas las columnas cargadas. Se procede a eliminar columnas que no serán últiles.

In [5]:
df.drop(columns = ["id", "title", "namespace", "revision_id", "revision_timestamp", "parent_revision", "revisor_username"], inplace = True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4132164 entries, 0 to 4132163
Data columns (total 3 columns):
 #   Column           Dtype  
---  ------           -----  
 0   revisor_id       float64
 1   revisor_ip       object 
 2   revisor_comment  object 
dtypes: float64(1), object(2)
memory usage: 94.6+ MB


Para esta consigna se tienen 3 tipos de grupos (los que hicieron una última contribución, >10 últimas contribuciones y >100 últimas contribuciones). Cada uno de estos grupos tiene 2 tipo de revisores: los que están registrados y los que no. Son mutuamente excluyentes (es decir, sí un usuario está registrado (i.e tiene revisor_id), no puede tener revisor_ip y viceversa), sin embargo puede suceder que haya usuarios que no tengan ni revisor_id ni revisor_ip, y usuarios que tengan ambas, así que primero se eliminaran las rows con estos grupos para limpiar el dataset. 

In [7]:
df.loc[df['revisor_id'].isnull() & df['revisor_ip'].isnull(), :]

,revisor_id,revisor_ip,revisor_comment
270100,NaN,NaN,enlaces


Se observa que la fila 270100 tiene un revisor que ni tiene revisor_id ni revisor_ip. Se procede a eliminarla.

In [8]:
df.loc[~df['revisor_id'].isnull() & ~df['revisor_ip'].isnull(), :]

,revisor_id,revisor_ip,revisor_comment


No hay ningún revisor que tenga revisor_id y revisor_ip al mismo tiempo.

In [9]:
df.drop(df.index[270100], inplace = True)

## Preparando el dataset
Ahora preparo el dataset para trabajarlo. Necesito la frecuencia de aparición de los revisores registrados y de los no registrados en una columna.

In [10]:
df['revisor_id'].value_counts()

5045504.0    417275
341614.0     374072
319121.0     112486
934283.0     108238
1797350.0     76523
              ...  
5518429.0         1
5518558.0         1
5518668.0         1
5518731.0         1
4503631.0         1
Name: revisor_id, Length: 44537, dtype: int64

In [11]:
df['revisor_ip'].value_counts()

80.29.25.39                              1074
82.159.73.41                             1023
83.240.186.98                             771
158.99.5.137                              460
186.101.157.58                            459
                                         ... 
191.95.175.129                              1
2806:1000:8102:7F:ED10:AB75:EDF0:6110       1
2806:104E:A:3AB4:7C4E:9347:CEAB:B0E3        1
181.176.96.114                              1
84.77.161.182                               1
Name: revisor_ip, Length: 107647, dtype: int64

In [12]:
dfRevisoresRegistrados = df.loc[df['revisor_ip'].isnull(), :] 
print(len(dfRevisoresRegistrados))
dfRevisoresRegistrados

3938877


,revisor_id,revisor_ip,revisor_comment
0,1708233.0,NaN,Así esta mejor
2,4642283.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...
3,5338508.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...
4,3615366.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...
5,689866.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...
...,...,...,...
4132159,474907.0,NaN,termino traducción y quito plantilla en desarr...
4132160,4032712.0,NaN,Crls9011 trasladó la página [[Manifestación de...
4132161,158577.0,NaN,Gabriel Solo trasladó la página [[Rocasaurus]]...
4132162,5429951.0,NaN,Términus trasladó la página [[Medalla Conmemor...


Al haber eliminado las columnas con revisor_ip null, se asegura que todas las filas tienen información en la columna revisor_id. Por lo tanto, hacer el count me va a dar la cantidad total de última edición que hizo un revisor con un determinado id. Dicha información la pongo en una nueva columna freq_id.

In [13]:
dfRevisoresRegistrados['freq_rev'] = dfRevisoresRegistrados.groupby('revisor_id')['revisor_id'].transform('count')
dfRevisoresRegistrados

,revisor_id,revisor_ip,revisor_comment,freq_rev
0,1708233.0,NaN,Así esta mejor,6937
2,4642283.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,3868
3,5338508.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,24169
4,3615366.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...,2905
5,689866.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...,831
...,...,...,...,...
4132159,474907.0,NaN,termino traducción y quito plantilla en desarr...,37975
4132160,4032712.0,NaN,Crls9011 trasladó la página [[Manifestación de...,184
4132161,158577.0,NaN,Gabriel Solo trasladó la página [[Rocasaurus]]...,1721
4132162,5429951.0,NaN,Términus trasladó la página [[Medalla Conmemor...,783


In [14]:
dfRevisoresNoRegistrados = df.loc[df['revisor_id'].isnull(), :] 
print(len(dfRevisoresNoRegistrados))
dfRevisoresNoRegistrados

193286


,revisor_id,revisor_ip,revisor_comment
1,NaN,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,Añadir contenido
24,NaN,2806:1000:8101:7AFD:4962:AE:51CB:EFFD,NaN
45,NaN,190.164.54.234,/* top */añadí chile a la lista de animales pr...
48,NaN,179.53.218.243,NaN
112,NaN,190.232.74.253,NaN
...,...,...,...
4132090,NaN,220.124.67.41,NaN
4132103,NaN,220.124.67.41,Página creada con «{{Ficha de serie de televis...
4132105,NaN,220.124.67.41,Página redirigida a [[EBS Global News]]
4132111,NaN,181.194.57.174,Página creada con «{{Infobox football club | ...


Lo mismo sucede con los revisores no registrados: al haberse quitado las filas con columnas revisor_id iguales a null, se asegura que la columnas revisor_ip tienen información, por lo que se puede hacer el count con la finalidad de antes.

In [15]:
dfRevisoresNoRegistrados['freq_rev'] = dfRevisoresNoRegistrados.groupby('revisor_ip')['revisor_ip'].transform('count')
print(len(dfRevisoresNoRegistrados))
dfRevisoresNoRegistrados

193286


,revisor_id,revisor_ip,revisor_comment,freq_rev
1,NaN,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,Añadir contenido,13
24,NaN,2806:1000:8101:7AFD:4962:AE:51CB:EFFD,NaN,2
45,NaN,190.164.54.234,/* top */añadí chile a la lista de animales pr...,2
48,NaN,179.53.218.243,NaN,1
112,NaN,190.232.74.253,NaN,1
...,...,...,...,...
4132090,NaN,220.124.67.41,NaN,7
4132103,NaN,220.124.67.41,Página creada con «{{Ficha de serie de televis...,7
4132105,NaN,220.124.67.41,Página redirigida a [[EBS Global News]],7
4132111,NaN,181.194.57.174,Página creada con «{{Infobox football club | ...,303


### Grupo de revisores con 1 última edición 

In [16]:
dfRevisoresRegistrados['revisor_id'].value_counts()


5045504.0    417275
341614.0     374072
319121.0     112486
934283.0     108238
1797350.0     76523
              ...  
5518429.0         1
5518558.0         1
5518668.0         1
5518731.0         1
4503631.0         1
Name: revisor_id, Length: 44537, dtype: int64

In [17]:
dfRevisoresRegistrados1Contribucion = dfRevisoresRegistrados.loc[dfRevisoresRegistrados['freq_rev'] == 1, :]
dfRevisoresRegistrados1Contribucion

,revisor_id,revisor_ip,revisor_comment,freq_rev
50,6179998.0,NaN,Unos pequeños detalles,1
166,6324944.0,NaN,he modificado un parrafo,1
184,153961.0,NaN,Deshecha la edición 6633628 de [[Special:Contr...,1
235,6058966.0,NaN,NaN,1
302,6018310.0,NaN,No es real que los vuelos de cabotaje sean de ...,1
...,...,...,...,...
4131723,6332400.0,NaN,NaN,1
4131771,6291332.0,NaN,completar cita libro,1
4131957,6331508.0,NaN,NaN,1
4132053,6252228.0,NaN,Agregar imágenes,1


In [18]:
dfRevisoresNoRegistrados1Contribucion = dfRevisoresNoRegistrados.loc[dfRevisoresNoRegistrados['freq_rev'] == 1, :]
dfRevisoresNoRegistrados1Contribucion

,revisor_id,revisor_ip,revisor_comment,freq_rev
48,NaN,179.53.218.243,NaN,1
112,NaN,190.232.74.253,NaN,1
117,NaN,173.237.243.28,/* Problemas de optimización */,1
137,NaN,45.71.202.51,NaN,1
140,NaN,62.57.63.181,NaN,1
...,...,...,...,...
4131076,NaN,85.48.30.196,/* Historia */,1
4131085,NaN,2A02:1205:506C:9900:4D80:C0D7:7364:F926,/* Enlaces y referencias externas */,1
4131752,NaN,187.144.180.20,/* Cuadro principal */,1
4131924,NaN,82.18.94.240,Página redirigida a [[Atomic Betty]],1


In [19]:
dfRevisores1Contribucion = dfRevisoresRegistrados1Contribucion.append(dfRevisoresNoRegistrados1Contribucion, ignore_index = True)
dfRevisores1Contribucion

,revisor_id,revisor_ip,revisor_comment,freq_rev
0,6179998.0,NaN,Unos pequeños detalles,1
1,6324944.0,NaN,he modificado un parrafo,1
2,153961.0,NaN,Deshecha la edición 6633628 de [[Special:Contr...,1
3,6058966.0,NaN,NaN,1
4,6018310.0,NaN,No es real que los vuelos de cabotaje sean de ...,1
...,...,...,...,...
114716,NaN,85.48.30.196,/* Historia */,1
114717,NaN,2A02:1205:506C:9900:4D80:C0D7:7364:F926,/* Enlaces y referencias externas */,1
114718,NaN,187.144.180.20,/* Cuadro principal */,1
114719,NaN,82.18.94.240,Página redirigida a [[Atomic Betty]],1


In [20]:
cantComentariosNulRevisores1Contribucion = len(dfRevisores1Contribucion.loc[dfRevisores1Contribucion['revisor_comment'].isnull(), :])
cantRevisores1ContribucionTotales = len(dfRevisores1Contribucion)
porcentaje1ContribucionSinComentario = cantComentariosNulRevisores1Contribucion / cantRevisores1ContribucionTotales
print(porcentaje1ContribucionSinComentario)

0.35215871549236843


## Grupo de revisores con más de 10 últimas ediciones

In [21]:
dfRevisoresRegistrados10MasContribuciones = dfRevisoresRegistrados.loc[dfRevisoresRegistrados['freq_rev'] > 10, :]
dfRevisoresRegistrados10MasContribuciones

,revisor_id,revisor_ip,revisor_comment,freq_rev
0,1708233.0,NaN,Así esta mejor,6937
2,4642283.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,3868
3,5338508.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,24169
4,3615366.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...,2905
5,689866.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...,831
...,...,...,...,...
4132159,474907.0,NaN,termino traducción y quito plantilla en desarr...,37975
4132160,4032712.0,NaN,Crls9011 trasladó la página [[Manifestación de...,184
4132161,158577.0,NaN,Gabriel Solo trasladó la página [[Rocasaurus]]...,1721
4132162,5429951.0,NaN,Términus trasladó la página [[Medalla Conmemor...,783


In [22]:
dfRevisoresNoRegistrados10MasContribuciones = dfRevisoresNoRegistrados.loc[dfRevisoresNoRegistrados['freq_rev'] > 10, :]
dfRevisoresNoRegistrados10MasContribuciones

,revisor_id,revisor_ip,revisor_comment,freq_rev
1,NaN,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,Añadir contenido,13
327,NaN,2806:261:40A:8797:5C51:8D37:199C:BA0F,NaN,23
473,NaN,2800:810:55D:5A4:D853:F635:E61:4225,/* Luminosidades típicas en candelas (cd) */,61
719,NaN,2001:1388:782B:1F61:C503:AAE8:2196:C9F2,Deshecha la edición 137966184 de [[Especial:Co...,18
862,NaN,179.6.217.59,/* Reparto */,20
...,...,...,...,...
4131982,NaN,2806:106E:15:4D73:D26:F175:CDEE:1953,Página creada con «{{Ficha de sencillo |nombre...,20
4131983,NaN,2806:106E:15:4D73:D26:F175:CDEE:1953,Página creada con «[[Categoría:Canciones por a...,20
4131990,NaN,2806:106E:15:4D73:D26:F175:CDEE:1953,NaN,20
4132111,NaN,181.194.57.174,Página creada con «{{Infobox football club | ...,303


In [23]:
dfRevisores10MasContribuciones = dfRevisoresRegistrados10MasContribuciones.append(dfRevisoresNoRegistrados10MasContribuciones, ignore_index = True)
dfRevisores10MasContribuciones

,revisor_id,revisor_ip,revisor_comment,freq_rev
0,1708233.0,NaN,Así esta mejor,6937
1,4642283.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,3868
2,5338508.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,24169
3,3615366.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...,2905
4,689866.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...,831
...,...,...,...,...
3913916,NaN,2806:106E:15:4D73:D26:F175:CDEE:1953,Página creada con «{{Ficha de sencillo |nombre...,20
3913917,NaN,2806:106E:15:4D73:D26:F175:CDEE:1953,Página creada con «[[Categoría:Canciones por a...,20
3913918,NaN,2806:106E:15:4D73:D26:F175:CDEE:1953,NaN,20
3913919,NaN,181.194.57.174,Página creada con «{{Infobox football club | ...,303


In [24]:
cantComentariosNulRevisores10MasContribuciones = len(dfRevisores10MasContribuciones.loc[dfRevisores10MasContribuciones['revisor_comment'].isnull(), :])
cantRevisores10MasContribucionesTotales = len(dfRevisores10MasContribuciones)
porcentaje10MasContribucionesSinComentario = cantComentariosNulRevisores10MasContribuciones / cantRevisores10MasContribucionesTotales
print(porcentaje10MasContribucionesSinComentario)

0.10140649236405129


## Grupo de revisores con más de 100 últimas ediciones

In [25]:
dfRevisoresRegistrados100MasContribuciones = dfRevisoresRegistrados.loc[dfRevisoresRegistrados['freq_rev'] > 100, :]
dfRevisoresRegistrados100MasContribuciones

,revisor_id,revisor_ip,revisor_comment,freq_rev
0,1708233.0,NaN,Así esta mejor,6937
2,4642283.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,3868
3,5338508.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,24169
4,3615366.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...,2905
5,689866.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...,831
...,...,...,...,...
4132159,474907.0,NaN,termino traducción y quito plantilla en desarr...,37975
4132160,4032712.0,NaN,Crls9011 trasladó la página [[Manifestación de...,184
4132161,158577.0,NaN,Gabriel Solo trasladó la página [[Rocasaurus]]...,1721
4132162,5429951.0,NaN,Términus trasladó la página [[Medalla Conmemor...,783


In [26]:
dfRevisoresNoRegistrados100MasContribuciones = dfRevisoresNoRegistrados.loc[dfRevisoresNoRegistrados['freq_rev'] > 100, :]
dfRevisoresNoRegistrados100MasContribuciones

,revisor_id,revisor_ip,revisor_comment,freq_rev
957,NaN,158.99.5.137,/* Usos */,460
1213,NaN,82.159.73.41,NaN,1023
2909,NaN,181.54.172.18,NaN,360
4106,NaN,190.161.191.134,/* Ciudades hermanadas */,366
4223,NaN,195.235.193.15,NaN,206
...,...,...,...,...
4131700,NaN,181.194.57.174,/* Enlaces externos */,303
4131713,NaN,181.194.57.174,/* Enlaces externos */,303
4131862,NaN,190.238.101.60,/* Enlaces externos */,236
4132111,NaN,181.194.57.174,Página creada con «{{Infobox football club | ...,303


In [27]:
dfRevisores100MasContribuciones = dfRevisoresRegistrados100MasContribuciones.append(dfRevisoresNoRegistrados100MasContribuciones, ignore_index = True)
dfRevisores100MasContribuciones

,revisor_id,revisor_ip,revisor_comment,freq_rev
0,1708233.0,NaN,Así esta mejor,6937
1,4642283.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,3868
2,5338508.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,24169
3,3615366.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...,2905
4,689866.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...,831
...,...,...,...,...
3734502,NaN,181.194.57.174,/* Enlaces externos */,303
3734503,NaN,181.194.57.174,/* Enlaces externos */,303
3734504,NaN,190.238.101.60,/* Enlaces externos */,236
3734505,NaN,181.194.57.174,Página creada con «{{Infobox football club | ...,303


In [28]:
cantComentariosNulRevisores100MasContribuciones = len(dfRevisores100MasContribuciones.loc[dfRevisores100MasContribuciones['revisor_comment'].isnull(), :])
cantRevisores100MasContribucionesTotales = len(dfRevisores100MasContribuciones)
porcentaje100MasContribucionesSinComentario = cantComentariosNulRevisores100MasContribuciones / cantRevisores100MasContribucionesTotales
print(porcentaje100MasContribucionesSinComentario)

0.09365573554956517


## Conclusiones
Se puede concluir que los usuarios que más hacen ediciones son más propensos a dejar un comentario de la revisión como buena práctica. 